In [ ]:
!pip install yfinance
!pip install finta
from finta import TA
import pandas as pd
import yfinance as yf
import glob

In [ ]:
# Function with a 'folder' and 'years' argument. Purpose is to download and save historic price data for each ticker as .csv and also in a dictionary.
def download_stock_price_data_and_save_to_a_dict(folder='', years=5):

  # 'tickers' is a pandas DataFrame object. 'pd.read_csv' function reads csv file from this website which contains all of the stock ticker symbols under it's "ACT Symbol" column.
  tickers = pd.read_csv('https://pkgstore.datahub.io/core/nyse-other-listings/nyse-listed_csv/data/3c88fab8ec158c3cd55145243fe5fcdf/nyse-listed_csv.csv')
  # 'symbols' is a list of symbols we made by taking all the unique symbols from the tickers .csv
  symbols = tickers['ACT Symbol'].unique()
  # empty dictionary to store all stock price DataFrames.
  price_data_dict = {}

  # loop where we go through each symbol 's' in the symbols list (first 20), and then do some things.
  for s in symbols[0:20]:
    try:

      # creates the DataFrame 'price_data_df' from the downloaded yf price data over the last years. auto_adjust accounts for stock splits and such. Takes in each stock symbol 's' one at at time.
      price_data_df = yf.download([s], auto_adjust=True, period=f'{str(years)}y')

      # If the length of the dataframe is over 100, then go ahead and do the iteration.
      if len(price_data_df) > 100:
        # make the columns lowercase, rename the index to 'datetime', and adds the 'price_data_df' to the 'price_data_dict' with the key being the current symbol 's'
        price_data_df.columns = price_data_df.columns.str.lower()
        price_data_df.index = price_data_df.index.rename('datetime')
        price_data_df.to_csv(f'{folder}{s}.csv')
        price_data_dict[s] = price_data_df

    # error catch
    except Exception as e:
      print(f'error with {s}')
      print(repr(e))
      continue
  
  # Allows for the 'price_data_dict' dictionary object to be accessed elsewhere in the code as a variable or input in another function.   
  return price_data_dict
  
# Download and saves price data to our price data dictionary.
price_data_dict = download_stock_price_data_and_save_to_a_dict()

In [4]:
# takes in 'price_data_df' DataFrames as an argument and transforms it to a new DataFrame later called 'entries'.
def create_entry_df(price_data_df:pd.DataFrame) -> pd.DataFrame:
  # Adding various columns to the original 'df'.
  price_data_df['EMA200'] = TA.EMA(price_data_df, period=200)
  price_data_df['EMA50'] = TA.EMA(price_data_df, period=50)
  price_data_df['ATR'] = TA.ATR(price_data_df)
  # entry is lazily done, it's just the opening price of the next row
  price_data_df['entry'] = price_data_df.shift(-1).open
  price_data_df['target'] = 0.0
  price_data_df['stop'] = 0.0
  price_data_df['risk:reward'] = 0.0
  price_data_df['target_pct'] = 0.0
  price_data_df['stop_pct'] = 0.0
  price_data_df['target_profit'] = 150
  price_data_df['position_size'] = 0.0

  # Conditions for long entry based on technical analysis indicators.
  c1 = price_data_df.close > price_data_df.EMA200
  c2 = (price_data_df.low < price_data_df.EMA50) & (price_data_df.close > price_data_df.EMA50) & (price_data_df.open > price_data_df.EMA50)
  long_entry_condition = (c1) & (c2)

  # Accesses and modifies all rows in 'df' where long_entry_condition is true, and modifies the 2nd input '' column in those rows to what is specified.
  price_data_df.loc[long_entry_condition, 'target'] = price_data_df.entry + (1.5*price_data_df.ATR)
  price_data_df.loc[long_entry_condition, 'stop'] = price_data_df.entry - price_data_df.ATR
  price_data_df.loc[long_entry_condition, 'risk:reward'] = (price_data_df.target - price_data_df.entry) / (price_data_df.entry - price_data_df.stop)
  price_data_df.loc[long_entry_condition, 'target_pct'] = (price_data_df.target - price_data_df.entry) / (price_data_df.entry)
  price_data_df.loc[long_entry_condition, 'stop_pct'] = -(price_data_df.entry - price_data_df.stop) / (price_data_df.entry)
  price_data_df.loc[long_entry_condition, 'position_size'] = price_data_df.target_profit / price_data_df.target_pct

  # Selects the subset of rows where the price data in DataFrame 'price_data_df' satisfy the long entry condition, and then creates a copy of the modified DataFrame denoted 'entries'.
  entries = price_data_df.loc[long_entry_condition][['entry', 'target', 'stop', 'risk:reward', 'target_pct', 'stop_pct', 'target_profit', 'position_size']].copy()

  # Allows for the 'entries' DataFrame to be accessed elsewhere in the code as a variable. 
  return entries

In [5]:
# Create empty list and dictionary for future use.
# Goal of this chunk: Needed to grab all the 'survivor' valid .csv files with actual data, so couldn't just grab every single symbol from the 'symbols' array to plug into the trade entries df's.
filepaths = []
trade_entries_dict = {}

# Uses the 'glob' module to iterate and find all files with .csv, for each file found, the filepath is appended to the filepaths empty list [ ].
for file in glob.glob('*.csv'):
  filepaths.append(file)

# Using a for loop, we iterate through each filepath in 'filepaths' list.
for path in filepaths:
  # Extracts the file's symbol name by splitting and parsing the path at the first index before the period, and saves it to the symbol variable.
  symbol = path.split('.')[0]
  # Reads the file into a DataFrame called 'symbols_df', with the index set to 'datetime'. The parse_dates makes it so the datetime index column is properly a datetime64[ns] object type.
  symbols_df = pd.read_csv(path, index_col='datetime', parse_dates=['datetime'])
  # Calls the 'create_entry_df' function on 'symbols_df' to generate an entirely new DataFrame called 'entry_df', which has the trade entries for a single symbol.
  entry_df = create_entry_df(symbols_df)
  # It adds each DataFrame 'entry_df' which was iterated through for every symbol, and adds it to 'trade_entries_dict', with the symbol name as the key, and the value is the entry trade data.
  trade_entries_dict[symbol] = entry_df

In [ ]:
trade_entries_dict

In [ ]:
price_data_dict

In [9]:
# merges the price data and trade entries dictionaries. It's shorthand for creating a new dictionary from existing ones.
# for each 'key' in price_data_dict, it makes new key-value pair in the new dict. Rows are aligned based on their common index.
price_and_trades_dict = {key: pd.concat([price_data_dict[key], trade_entries_dict[key]], axis=1) for key in price_data_dict}

In [ ]:
price_and_trades_dict

In [20]:
# Function with purpose to create new dictionary which houses the trade results, with an input of the price_and_trades_dict dictionary.
def create_trade_results_dict(price_and_trades_dict):
    trade_results_dict = {}
    # Loop over items (view object key:value pairs) in 'price_and_trades_dict', going through each tuple one by one, where the key is 'symbol' and the value are the data frames 'trades_df' 
    for symbol, trades_df in price_and_trades_dict.items():
        # Initiating empty list and setting the trade number, which will be the index of our new data frames, to 1.
        trade_results = []
        trade_num = 1
        # Iterate rows in the data frame, yielding pairs of index and row data as a series. 'idx' is index which is the row label, and 'row' represents data in the current row as a series.
        for idx, row in trades_df.iterrows():
            # Check if the value of 'target' column in current row is > 0. (We are checking if a valid trade has taken place by making sure 'target' is not NaN basically)
            if row['target'] > 0:
                # Saves the index of the next row to 'entry_idx' variable. This accounts for the proper 'entry_date' being the next trading days open after a trade is triggered.
                entry_idx = trades_df.index.get_loc(idx) + 1
                # Check if the saved index is less than the length of the data frame. Thie ensures that the next row exists within the DataFrame, preventing errors.
                if entry_idx < len(trades_df):
                    # Sets the entry_date to the index of the entry.
                    entry_date = trades_df.index[entry_idx]
                    # Finds the exit row by starting at the 'entry_date' and onward due to the empty value after the ':'. Then it applies the lambda function to each row to check and see when the 'target' or 'stop' value from the original row is crossed by a future 'high' or 'low' value. 'idxmax()' returns index of the first true value since true is considered greater than false. So the result of this is the index of the exit row is assigned to 'exit_row_idx'.
                    exit_row_idx = trades_df.loc[entry_date:].apply(lambda x: x['high'] >= row['target'] or x['low'] <= row['stop'], axis=1).idxmax()
                    # exit_price is target or stop depending on which is hit.
                    exit_price = row['target'] if trades_df.loc[exit_row_idx, 'high'] >= row['target'] else row['stop']
                    # exit_date is the index of the exit row.
                    exit_date = exit_row_idx
                    # Simple calculations
                    days_in_trade = (exit_date - entry_date).days
                    win_loss = 1 if trades_df.loc[exit_row_idx, 'high'] >= row['target'] else -1
                    p_l = 150 if trades_df.loc[exit_row_idx, 'high'] >= row['target'] else -100
                    # Appends the desired row values to the initially empty 'trade_results' list, and then ticks forward the 'trade_num' to start the process again.
                    trade_results.append([trade_num, entry_date, row['entry'], row['target'], row['stop'], exit_price, exit_date, days_in_trade, row['position_size'], win_loss, p_l])
                    trade_num += 1
        # Creates a data frame which houses the list of values from the 'trade_results' list, with the following strings as the column headers.
        trade_results_df = pd.DataFrame(trade_results, columns=['trade_num', 'entry_date', 'entry', 'target', 'stop', 'exit_price', 'exit_date', 'days_in_trade', 'position_size', 'win_loss', 'p_l'])
        # Sets index of the data frame to the trade number
        trade_results_df.set_index('trade_num', inplace=True)
        #Adds the DataFrame to the dictionary with the current symbol as the key, storing the trade results for each symbol in the dictionary.
        trade_results_dict[symbol] = trade_results_df
    # Returns entire dictionary with all the symbols after the loop is finished, so we can use it elsewhere in the code.
    return trade_results_dict
# Produces the 'trade_results_dict' based on the 'price_and_trades_dict' data, storing the entire dictionary to the 'trade_results_dict' variable.
trade_results_dict = create_trade_results_dict(price_and_trades_dict)

In [21]:
trade_results_dict

{'A':           entry_date       entry      target        stop  exit_price  \
 trade_num                                                              
 1         2018-05-09   64.922352   66.764065   63.694544   66.764065   
 2         2018-06-18   63.091654   64.760965   61.978780   61.978780   
 3         2018-08-17   63.244033   65.168842   61.960827   65.168842   
 4         2018-11-08   65.807323   68.568939   63.966246   63.966246   
 5         2018-11-21   65.487911   68.233972   63.657203   68.233972   
 6         2019-01-09   66.173811   69.498242   63.957524   69.498242   
 7         2019-06-21   71.112158   72.965384   69.876674   72.965384   
 8         2019-06-25   71.160765   72.966072   69.957228   72.966072   
 9         2019-06-26   70.626022   72.441745   69.415540   72.441745   
 10        2019-06-28   71.228828   73.033089   70.025988   73.033089   
 11        2019-10-21   73.121142   75.451162   71.567795   75.451162   
 12        2019-10-28   73.912119   75.970112 